In [1]:
%run "./model_training.ipynb"

In [2]:
# 读取example_data.ipynb中生成的数据
ds_block = xr.open_dataset(
   "Data/block.nc")
ds_full = xr.open_dataset(
   "Data/full.nc")
# 将生成的数据转化为用于神经网络训练的数据集格式
BLOCK_SIZE, BATCH_SIZE = 2, 16
dens_full, disp_slice, outputs = extract_data(ds_full, ds_block)
dataset = tf.data.Dataset.from_tensor_slices(({"dens_input": dens_full, "disp_input": disp_slice}, outputs))
train_dataset, _, test_dataset = get_dataset(dataset, dens_full.shape[0], batch_size=BATCH_SIZE)

# 初始化神经网络模型
num_of_layers_cnn, num_of_layers_fnn = 10, 10
cnn = cnn_model(num_of_layers=num_of_layers_cnn)
fnn = fnn_model(block_size=BLOCK_SIZE, num_of_layers=num_of_layers_fnn)

In [3]:
# 训练神经网络并保存至"Model/{casename}"文件夹下
SAVE_MODEL = True
test_losses, val_losses = custom_train(block_size=BLOCK_SIZE,cnn_model=cnn,fnn_model=fnn,\
                        loss_function=tf.keras.losses.MeanAbsoluteError(),\
                        num_epoch=50,optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),\
                        batched_train=train_dataset, batched_test=test_dataset, 
                        num_of_layers_1=num_of_layers_cnn, num_of_layers_2=num_of_layers_fnn,
                        save_model=SAVE_MODEL, 
                        casename='example')

epoch: 0 loss: 0.11942468583583832 val_loss: 0.27034232020378113
epoch: 1 loss: 0.08815663307905197 val_loss: 0.09534742683172226
epoch: 2 loss: 0.0743429884314537 val_loss: 0.08027273416519165
epoch: 3 loss: 0.07088963687419891 val_loss: 0.07251618802547455
epoch: 4 loss: 0.06744831800460815 val_loss: 0.06912465393543243
epoch: 5 loss: 0.06615634262561798 val_loss: 0.06873439997434616
epoch: 6 loss: 0.0641382560133934 val_loss: 0.06487612426280975
epoch: 7 loss: 0.06272776424884796 val_loss: 0.06317955255508423
epoch: 8 loss: 0.06208276376128197 val_loss: 0.06285544484853745
epoch: 9 loss: 0.06090536713600159 val_loss: 0.059941988438367844
epoch: 10 loss: 0.05962803214788437 val_loss: 0.058155518025159836
epoch: 11 loss: 0.05871041119098663 val_loss: 0.056936606764793396
epoch: 12 loss: 0.05805955454707146 val_loss: 0.05760783329606056
epoch: 13 loss: 0.057444337755441666 val_loss: 0.05832236260175705
epoch: 14 loss: 0.056453555822372437 val_loss: 0.05583152920007706
epoch: 15 loss: 0